In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time
import sqlite3
import re


In [ ]:
df = pd.read_csv('data/Bird_strikes.csv')
df.head().T

In [ ]:
def clean_airport_name(airport_name):
    airport_name = airport_name.strip()    
    airport_name = airport_name.replace("ARPT", "Airport")
    airport_name = airport_name.replace("AIRPORT", "Airport")
    airport_name = airport_name.replace("INTL", "International")
    airport_name = airport_name.replace("RGNL", "Regional")
    airport_name = airport_name.replace("FLD", "Field")
    airport_name = airport_name.replace("AFB", "Air Force Base")
    airport_name = airport_name.replace("NAS", "Naval Air Station")
    airport_name = airport_name.replace("SPB", "Seaplane Base")
    airport_name = airport_name.replace("/", " ")
    airport_name = airport_name.replace("-", " ")
   

    return airport_name.strip()


df['AirportName'] = df['AirportName'].apply(clean_airport_name)

connection = sqlite3.connect('airport_coordinates.db')
cursor = connection.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS coordinates (
        airport_name TEXT PRIMARY KEY,
        latitude REAL,
        longitude REAL
    )
''')
connection.commit()


def get_coordinates(airport_name):
    
    cursor.execute("SELECT latitude, longitude FROM coordinates WHERE airport_name=?", (airport_name,))
    result = cursor.fetchone()
    if result:
        return result[0], result[1]

    geolocator = Nominatim(user_agent="BirdStrikeAnalyseProjekt_M_Hillen")  #
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    try:
        location = geocode(airport_name, timeout=10)  
        if location:
            coordinates = (location.latitude, location.longitude)
            
            cursor.execute("INSERT OR REPLACE INTO coordinates (airport_name, latitude, longitude) VALUES (?, ?, ?)",
                           (airport_name, location.latitude, location.longitude))
            connection.commit()
            return coordinates
        else:
            
            cursor.execute("INSERT OR REPLACE INTO coordinates (airport_name, latitude, longitude) VALUES (?, NULL, NULL)",
                           (airport_name, ))
            connection.commit()
            return None, None
    except Exception as e:
        print(f"Fehler beim Geocoding von {airport_name}: {e}")
        
        cursor.execute("INSERT OR REPLACE INTO coordinates (airport_name, latitude, longitude) VALUES (?, NULL, NULL)",
                       (airport_name,))
        connection.commit()
        return None, None



unique_airport_names = df['AirportName'].unique()

airport_coordinates = {}

start_time = time.time()
for airport_name in unique_airport_names:
    airport_coordinates[airport_name] = get_coordinates(airport_name)
end_time = time.time()

print("Time needed for coodinate creation: " + str(end_time - start_time))

df['Latitude'] = df['AirportName'].map(lambda x: airport_coordinates[x][0])
df['Longitude'] = df['AirportName'].map(lambda x: airport_coordinates[x][1])




unmatched_airports = df[df['Latitude'].isnull()]['AirportName'].unique()

def clean_problematic_airport_name(airport_name):
    # Spezielle Bereinigungen für problematische Flughäfen
    airport_name = airport_name.replace("MUNI", "Municipal")
    airport_name = airport_name.replace("NATL", "National")
    airport_name = airport_name.replace("COUNTY", "")
    airport_name = airport_name.replace("CNTY", "")
    airport_name = airport_name.replace("MC ", "MC")

    airport_name = re.sub(r'\b\w{2}\b', '', airport_name)
    if "Airport" not in airport_name:
        airport_name += " Airport"
    # Entfernen einzelner Buchstaben (mit oder ohne Punkt)
    airport_name = re.sub(r'\b[A-Za-z]\.?(?=\s|$)\b', '', airport_name)

    # Ersetzen von mehrfachen Leerzeichen durch einzelne Leerzeichen
    airport_name = re.sub(' +', ' ', airport_name)
    
    # Fügen Sie hier weitere spezifische Ersetzungen hinzu
    return airport_name.strip()

def get_coordinates_for_problematic(airport_name):
    cleaned_name = clean_problematic_airport_name(airport_name)
    coords = get_coordinates(cleaned_name)
    if coords == (None, None):
        pass
    return coords

for airport in unmatched_airports:
    coords = get_coordinates_for_problematic(airport)
    if coords != (None, None):
        airport_coordinates[airport] = coords
      

df['Latitude'] = df['AirportName'].map(lambda x: airport_coordinates.get(x, (None, None))[0])
df['Longitude'] = df['AirportName'].map(lambda x: airport_coordinates.get(x, (None, None))[1])



connection.close()



In [ ]:
df['Latitude'].nunique()

In [ ]:
df['Latitude'].isna().sum()

In [ ]:
df.head().T

In [ ]:
unm_airports = df[df['Latitude'].isnull()]['AirportName'].unique()

print(len(unm_airports))
print((unm_airports))

In [ ]:
manual_airport_corrections = {
    'NEW CASTLE COUNTY': (39.7212, -75.6053),
    'Naval Air StationHVILLE International': (36.2317, -86.6783),
    'ASPEN PITKIN CO SARD': (39.2233, -106.8683),
    'BARKSDALE AIR FORCE BASE Airport': (32.5067, -93.6633),
    'DETROIT METRO WAYNE COUNTY Airport': (42.2125, -83.3533),
    'METRO OAKLAND International': (37.7217, -122.2200),
    'FORT SMITH MUNICIPAL Airport': (35.3667, -94.3667),
    'MCCARREN International': (36.0800, -115.1522),
    'BALTIMORE WASH International': (39.1755, -76.6683),
    'PANAMA CITY   BAY COUNTY International': (30.3500, -85.7967),
    'WAUKEGAN REGIONAL Airport': (42.4208, -87.8567),
    'CHARLESTON Air Force Base International Airport': (32.8964, -80.0405),
    'THE EASTERN IOWA Airport': (41.8869, -91.7119),
    'SMITH REYOLDS Airport': (36.1267, -80.2217),
    'TYLER POUNDS FIELD': (32.3500, -95.4000),
    'LONG ISLAND MAC ARTHUR': (40.7850, -73.0767),
    'ELLINGTON FIELD Airport': (29.6067, -95.1717),
    'GAINESVILLE REG Airport': (29.6900, -82.2717),
    'LA CROSSE MUNICIPAL Airport': (43.8789, -91.2569),
    'BUCHOLZ AAF MARSHALL': (28.7958, -96.1867),
    'MALCOLM MCKINNON Airport': (30.4933, -84.9833),
    'MANaval Air StationSAS REGIONAL Airport': (32.6233, -97.6208),
    'LUIS MUNOZ MARIN International  SAN JUAN': (18.4394, -66.0017),
    'TORONTO LESTER B. PEARSON International': (43.6772, -79.6306),
    'BATON ROUGE METRO': (30.5333, -91.1500),
    'WILLOUGHBY LOST NATION MUNI Airport': (41.6383, -81.4100),
    'EASTERWOOD FIELD Airport': (30.5889, -96.3639),
    'KLAMATH FALLS International': (42.1667, -121.7167),
    'AKRON CANTON MUNICIPAL': (40.7250, -81.4439),
    'BROWNSVL SO PADRE IS': (25.9067, -97.4258),
    'MOBILE DOWNTOWN Airport': (30.6917, -88.0681),
    'EPHRAIM  FISH CREEK': (45.1667, -87.2167),
    'LONG BEACH DAUGH Field': (33.8178, -118.1517),
    'TEXARKANA WEBB FIELD': (33.4500, -94.0481),
    'LAKELAND LINDER REGIONAL': (27.9881, -82.0183),
    'BOEING FIELD K C INT': (47.5267, -122.3033),
    'JIM HOGG COUNTY AROT': (26.8500, -98.8833),
    'EASTON NEWMAN FIELD Airport': (38.7669, -76.0661),
    'SELAWICK Airport': (67.6500, -160.0000),
    'DE KALB TAYLOR MUNICIPAL Airport': (41.9383, -88.7350),
    'EL MONTE Airport': (34.0700, -118.0267),
    'ELMIRA CORNING MUNICIPAL': (42.1589, -76.7806),
    'BOISE AIR TERMINAL GOWEN FIELD': (43.5644, -116.2228),
    'USAF ACADEMY BULLSEYE AUX AIRSTRIP': (39.0067, -104.8400),
    'GLASGOW International WOKAL FIELD': (48.2194, -97.6267),
    'ELK GROVE Airport': (38.4167, -121.4167),
    'FERANDINA BEACH MUNICIPAL Airport': (30.6019, -81.4558),
    'BROOKINGS MUNICIPAL Airport': (44.3083, -96.7983),
    'ROGERS MUNICIPAL Airport CARTER FIELD': (36.3333, -94.0667),
    'CAPE GIRARDEAU MUNICIPAL': (37.2258, -89.5708),
    'DEMOPOLIS MUNICIPAL Airport': (32.5000, -87.8333),
    'BERZ MACOMB Airport': (42.6358, -82.9203),
    'HOUSTON GULF Airport': (29.6000, -95.2833),
    'MORAN FIELD Airport': (35.2261, -106.9064),
    'MANSFIELD LAHM MUNICIPAL': (40.7833, -82.5167),
    'CAPT FEAR REGIONAL JETPORT': (34.0833, -77.9000),
    'MERRILL C MEIGS Airport': (41.8611, -87.6064),
    'TERRE HAUTE International Airport HULMAN FIELD': (39.4628, -87.3069),
    'KINSTON REGL JETPORT': (35.3333, -77.6000),
    'RALEIGH EXEC JETPORT AT SANFORD LEE CNTY Airport': (35.5469, -79.1367),
    'FAYETTVILLE MUNICIPAL': (36.0000, -94.3000),
    'ANNISTON METROPOLITAN Airport': (33.5892, -85.8581),
    'JOHNSTON ATOLL Airport': (16.7442, 169.5208),
    'ROSEAU MUNI Airport RUDY BILLBERG Field': (48.8833, -95.7667),
    'GRIFFING SANDUSKY Airport': (41.4467, -82.6658),
    'SIDNEY MUNICIPAL AR[T': (40.2931, -84.1478),
    'MCCORD FIELD Airport': (47.1242, -122.4833),
    'PORT AU PRINCE International': (18.5767, -72.2917),
    'ADIRONDAK REGIONAL Airport': (44.4469, -73.9906),
    'ORANJESTAD REINA BEA': (12.5000, -70.0167),
    'QUINCY REGIONAL Airport   BALDWIN Field': (39.9417, -91.1931),
    'SALINaval Air Station MUNICIPAL Airport': (38.7906, -97.6656),
    'JASPER COUNTY Airport BELL FIELD': (30.4000, -94.2500),
    'COLLIN COUNTY Regional Airport AT MCKINNEY': (33.2900, -96.6100),
    'FAIRMONT MUNI FRANKMAN Field': (43.6419, -94.4586),
    'MCCOOK MUNICIPAL Airport': (40.2000, -100.6500),
    'SPRINGERVILLE MUNICIPAL': (34.2500, -109.2000),
    'OGDENSBURG International Airport': (44.6956, -75.4658),
    'VIRACOPOS CAMPINaval Air Station International Airport': (-23.4333, -47.1333),
    'PONCA CITY MUNICIPAL Airport': (36.7167, -97.0833),
    'M GRAHAM CLARK   TANEY COUNTY Airport': (36.8667, -93.2000),
    'MARCH ARB Airport': (33.8806, -117.2694),
    'CLAYTON COUNTY Airport  TARA FIELD': (33.5833, -84.3833),
    'GRANTLEY ADAMS BARBA': (13.0758, -59.4875),
    'DICKINSON THEODORE ROOSEVELT Regional': (46.7833, -102.7833),
    'JOSE MARIA CORDOVE Airport': (6.1667, -75.4167),
    'DAKAY YOFF LEOPOLD SEDAR SENGHOR': (14.7333, -17.4833),
    'HEBER CITY MUNICIPAL': (40.6467, -111.4017),
    'NORTH PALM BEACH CNTY GEN AVTN': (26.8500, -80.1833),
    'RIYAHD AIR BASE': (24.9500, 46.7000),
    'BOLINGBROOKS CLOW International': (41.6833, -88.1167),
    'SCOTT Air Force Base MIDAMERICA': (38.5400, -89.8500),
    'SAN PEDRO SULA LA MESA': (15.4500, -87.9167),
    'ROCK HILL (YORK CO) BRYANT Field': (34.9667, -81.0333),
    'CRAIG MUNICIPAL Airport': (30.2667, -81.5167),
    'ALPENA COUNTY REG': (45.0667, -83.5667),
    'WASHINGTON EXECUTIVE Airport HYDE FIELD': (38.7167, -76.8667),
    'GREAT BENT MUNICIPAL': (38.3500, -98.7667),
    'MARCO POLO International': (45.5053, 12.3519),
    'SHALZ FIELD Airport': (34.1833, -118.0667),
    'IXTAPA ZIHUATENEJO': (17.6500, -101.4833),
    'GEORGETOWN MUNICIPAL Airport': (30.6667, -97.6667),
    'DEVILS LAKE REGIONAL Airport': (48.1167, -98.9000),
    'GWINNER ROGER MELROE FIELD Airport': (46.4000, -97.6000),
    'CHEVRON USA INC SEAPLANE BASE': (61.2167, -149.9000),
    'LAWRENCE MUNICIPAL  Airport (KS)': (38.9500, -95.2000),
    'MC KELLER Field Airport': (32.6767, -94.7381),
    'RAMSTEIN AIR FORCE BASE': (49.4431, 7.6031),
    'THE FLORIDA KEYS MARATHON': (24.7333, -81.0500),
    'CRAWFORDSVILLE MUNI Airport': (40.0500, -86.9167),
    'MERRIT ISLAND Airport': (28.3667, -80.6667)
}

In [ ]:
def get_coordinates(airport_name):
    return manual_airport_corrections.get(airport_name, (None, None))

# Nur Zeilen ohne Koordinaten aktualisieren
mask = df['Latitude'].isnull() | df['Longitude'].isnull()
df.loc[mask, ['Latitude', 'Longitude']] = df.loc[mask, 'AirportName'].map(get_coordinates).tolist()


In [ ]:
df['Latitude'].isna().sum()

In [ ]:
df.describe()


In [ ]:
df.dtypes